In [2]:
import pyspark

In [3]:
pyspark.__version__

'3.5.0'

In [4]:
pyspark.__file__

'/usr/local/Cellar/apache-spark/3.5.0/libexec/python/pyspark/__init__.py'

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/24 09:12:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/02/24 09:13:08 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [8]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2024-02-23 19:07:53--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.195.240, 52.217.122.144, 54.231.198.208, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.195.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2024-02-23 19:07:54 (129 MB/s) - ‘taxi+_zone_lookup.csv’ saved [12322/12322]



In [10]:
!head -n 6 taxi+_zone_lookup.csv

"LocationID","Borough","Zone","service_zone"
1,"EWR","Newark Airport","EWR"
2,"Queens","Jamaica Bay","Boro Zone"
3,"Bronx","Allerton/Pelham Gardens","Boro Zone"
4,"Manhattan","Alphabet City","Yellow Zone"
5,"Staten Island","Arden Heights","Boro Zone"


In [18]:
df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv',inferSchema=True)

!wc -l taxi+_zone_lookup.csv

     266 taxi+_zone_lookup.csv


In [19]:
df.schema

StructType([StructField('LocationID', IntegerType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [20]:
df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [13]:
df.write.parquet('zones')

In [20]:
!ls -lh

total 312
-rw-r--r--  1 air  staff   7.7K Feb  8 15:30 03_test.ipynb
-rw-r--r--  1 air  staff    18K Feb  8 15:30 04_pyspark.ipynb
-rw-r--r--  1 air  staff    32K Feb  8 15:30 05_taxi_schema.ipynb
-rw-r--r--  1 air  staff   8.6K Feb  8 15:30 06_spark_sql.ipynb
-rw-r--r--  1 air  staff   2.5K Feb  8 15:30 06_spark_sql.py
-rw-r--r--  1 air  staff   2.6K Feb  8 15:30 06_spark_sql_big_query.py
-rw-r--r--  1 air  staff   7.5K Feb  8 15:30 07_groupby_join.ipynb
-rw-r--r--  1 air  staff    11K Feb  8 15:30 08_rdds.ipynb
-rw-r--r--  1 air  staff   4.3K Feb  8 15:30 09_spark_gcs.ipynb
-rw-r--r--  1 air  staff   2.9K Feb  8 15:30 cloud.md
-rwxr-xr-x  1 air  staff   544B Feb  8 15:30 download_data.sh
-rw-r--r--  1 air  staff    16K Feb  8 15:30 homework.ipynb
-rw-r--r--  1 air  staff    12K Aug 17  2016 taxi+_zone_lookup.csv
drwxr-xr-x  6 air  staff   192B Feb 23 19:10 zones


In [24]:
# !head zones/part-00000-504af732-5ed0-|4bdf-a92f-5ba857db536b-c000.snappy.parquet

df = spark.read \
    .option( "inferSchema",True) \
    .parquet('zones/part-00000-504af732-5ed0-4bdf-a92f-5ba857db536b-c000.snappy.parquet')

!wc -l zones/part-00000-504af732-5ed0-4bdf-a92f-5ba857db536b-c000.snappy.parquet

      18 zones/part-00000-504af732-5ed0-4bdf-a92f-5ba857db536b-c000.snappy.parquet


In [25]:
df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly